# ETL project:

    Our project that we went with was looking at NBA players using the Kaggle data source.  Inside the Kaggle data source we found the information we were looking to research, but it was in 3 different excel files: Player, Player Data, and Season Stats.  The goal was to extract the 3 files by converting them into CSV files based on the information we wanted to be returned, write our query through Jupiter Notebook, and create a DF using SQL.

    Getting started we: import pandas as pd, from sqlalchemy import create engine, import pymysql, pymysql. install_as_MySQLdb() to read in our CSV data and then push what we wanted returned to MySQL to a data frame.  There was a lot of columns in the information, so we picked out the once that we wanted in our database at the end (Player Name, id, Year, Team, Games Played, FG%, Free Throw %, Total Assist, and Total Points.)  Once we uploaded the Players and Season Stats CSV file, we ran into a few issues that needed to be CLEANED.

    The first issue we had was that some of the players played on multiple teams with in the same season so it caused an error because the player was the primary key.  What we did was make the “id” the primary key so that it could read through the data even with the duplicate names and the players who played with different teams their “id” number assigned stayed the same.

    The second issue we had was in the Players CSV file "college" was spelled "collage". What we had to was transform and rename the columns according to what we wanted the data to look like.

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import pymysql 
pymysql.install_as_MySQLdb()

In [2]:
Players = "../Desktop/nba-players-stats/Players.csv"
Players_df = pd.read_csv(Players)
Players_df.head()

,id,Player,height,weight,collage,born,birth_city,birth_state
0,0,Curly Armstrong,180.0,77.0,Indiana University,1918.0,NaN,NaN
1,1,Cliff Barker,188.0,83.0,University of Kentucky,1921.0,Yorktown,Indiana
2,2,Leo Barnhorst,193.0,86.0,University of Notre Dame,1924.0,NaN,NaN
3,3,Ed Bartels,196.0,88.0,North Carolina State University,1925.0,NaN,NaN
4,4,Ralph Beard,178.0,79.0,University of Kentucky,1927.0,Hardinsburg,Kentucky


In [3]:
Season_Stats = "../Desktop/nba-players-stats/Seasons_Stats.csv"
Season_Stats_df = pd.read_csv(Season_Stats)
Season_Stats_df.head()

,id,Year,Player,Pos,Age,Tm,G,GS,MP,PER,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,0,1950.0,Curly Armstrong,G-F,31.0,FTW,63.0,NaN,NaN,NaN,...,0.71,NaN,NaN,NaN,176.0,NaN,NaN,NaN,217.0,458.0
1,1,1950.0,Cliff Barker,SG,29.0,INO,49.0,NaN,NaN,NaN,...,0.71,NaN,NaN,NaN,109.0,NaN,NaN,NaN,99.0,279.0
2,2,1950.0,Leo Barnhorst,SF,25.0,CHS,67.0,NaN,NaN,NaN,...,0.70,NaN,NaN,NaN,140.0,NaN,NaN,NaN,192.0,438.0
3,3,1950.0,Ed Bartels,F,24.0,TOT,15.0,NaN,NaN,NaN,...,0.56,NaN,NaN,NaN,20.0,NaN,NaN,NaN,29.0,63.0
4,4,1950.0,Ed Bartels,F,24.0,DNN,13.0,NaN,NaN,NaN,...,0.55,NaN,NaN,NaN,20.0,NaN,NaN,NaN,27.0,59.0


In [4]:
new_players_df= Players_df[['Player', 'id','collage']].copy()
new_players_df.head()

,Player,id,collage
0,Curly Armstrong,0,Indiana University
1,Cliff Barker,1,University of Kentucky
2,Leo Barnhorst,2,University of Notre Dame
3,Ed Bartels,3,North Carolina State University
4,Ralph Beard,4,University of Kentucky


In [5]:
# Create a filtered dataframe from specific columns
new_players = ["Player","id", "collage"]
new_players_transformed= new_players_df[new_players].copy()

# Rename the column headers
new_players_transformed = new_players_transformed.rename(columns={"Player": "Player_Name",
                                                                  "id": "id",
                                                                  "collage": "college"})

# Clean the data by dropping duplicates and setting the index
new_players_transformed.drop_duplicates(inplace=True)
new_players_transformed.set_index('Player_Name', inplace=True)

new_players_transformed.head()

,id,college
Player_Name,,
Curly Armstrong,0,Indiana University
Cliff Barker,1,University of Kentucky
Leo Barnhorst,2,University of Notre Dame
Ed Bartels,3,North Carolina State University
Ralph Beard,4,University of Kentucky


In [6]:
New_Season_Stats_df = Season_Stats_df[['Player','id','Year', 'Tm', 'G', 'FG%', 'FT%', 'AST', 'PTS']].copy()
New_Season_Stats_df.head()

,Player,id,Year,Tm,G,FG%,FT%,AST,PTS
0,Curly Armstrong,0,1950.0,FTW,63.0,0.28,0.71,176.0,458.0
1,Cliff Barker,1,1950.0,INO,49.0,0.37,0.71,109.0,279.0
2,Leo Barnhorst,2,1950.0,CHS,67.0,0.35,0.70,140.0,438.0
3,Ed Bartels,3,1950.0,TOT,15.0,0.26,0.56,20.0,63.0
4,Ed Bartels,4,1950.0,DNN,13.0,0.26,0.55,20.0,59.0


In [7]:
# Create a filtered dataframe from specific columns
New_Stats = ["Player","id","Year", "Tm", "G", "FG%", "FT%", "AST", "PTS"]
New_Stats_transformed= New_Season_Stats_df[New_Stats].copy()

# Rename the column headers
New_Stats_transformed = New_Stats_transformed.rename(columns={"Player": "Player_Name",
                                                              "id": "id",
                                                              "Year": "Year",
                                                              "Tm": "Team",
                                                              "G": "Games_Played",
                                                              "FG%": "Field_Goal_Percentage",
                                                              "FT%": "Free_Throw_Percentage",
                                                              "AST": "Total_Assists",
                                                              "PTS": "Total_Points"})

# Clean the data by dropping duplicates and setting the index
New_Stats_transformed.drop_duplicates(inplace=True)
New_Stats_transformed.set_index("Player_Name", inplace=True)

New_Stats_transformed.head()

#output = df.to_string(formatters={
 #   'var1': '{:,.2f}'.format,
  #  'var2': '{:,.2f}'.format,
   #'var3': '{:,.2%}'.format
#})
#print(output)

#New_Stats_transformed.Field_Goal_Percentage = (New_Stats_transformed.Field_Goal_Percentage * 100).astype(str) + ".2f"
#New_Stats_transformed.Free_Throw_Percentage = (New_Stats_transformed.Free_Throw_Percentage * 100).astype(str) + ".2f"
#New_Stats_transformed.round({"Free_Throw_Percent": 2, "Field_Goal_Percent": 2})
#New_Stats_transformed.head()

,id,Year,Team,Games_Played,Field_Goal_Percentage,Free_Throw_Percentage,Total_Assists,Total_Points
Player_Name,,,,,,,,
Curly Armstrong,0,1950.0,FTW,63.0,0.28,0.71,176.0,458.0
Cliff Barker,1,1950.0,INO,49.0,0.37,0.71,109.0,279.0
Leo Barnhorst,2,1950.0,CHS,67.0,0.35,0.70,140.0,438.0
Ed Bartels,3,1950.0,TOT,15.0,0.26,0.56,20.0,63.0
Ed Bartels,4,1950.0,DNN,13.0,0.26,0.55,20.0,59.0


In [8]:
#output = df.to_string(formatters={
#   ‘var1’: ‘{:,.2f}’.format,
 #  ‘var2’: ‘{:,.2f}’.format,
  #‘var3’: ‘{:,.2%}’.format

#New_Stats_transformed['Field_Goal_Percentage'] = New_Stats_transformed['Field_Goal_Percentage'].map('{:.0%}'.format)
#New_Stats_transformed['Free_Throw_Percentage'] = New_Stats_transformed['Free_Throw_Percentage'].map('{:.0%}'.format)
#New_Stats_transformed.head()

In [9]:
New_Stats_transformed.head()

,id,Year,Team,Games_Played,Field_Goal_Percentage,Free_Throw_Percentage,Total_Assists,Total_Points
Player_Name,,,,,,,,
Curly Armstrong,0,1950.0,FTW,63.0,0.28,0.71,176.0,458.0
Cliff Barker,1,1950.0,INO,49.0,0.37,0.71,109.0,279.0
Leo Barnhorst,2,1950.0,CHS,67.0,0.35,0.70,140.0,438.0
Ed Bartels,3,1950.0,TOT,15.0,0.26,0.56,20.0,63.0
Ed Bartels,4,1950.0,DNN,13.0,0.26,0.55,20.0,59.0


In [10]:
connection_string = ("root:Men15ard@localhost:3306/nba_db")
engine = create_engine(f'mysql://{connection_string}')

In [11]:
# Confirm tables
engine.table_names()

['Players', 'Season_Stats']

In [12]:
New_Stats_transformed.to_sql(name='Season_Stats', con=engine, if_exists='append', index=True)

In [13]:
new_players_transformed.to_sql(name='Players', con=engine, if_exists='append', index=True)